<a href="https://colab.research.google.com/github/Hj-lh/Fine-tune-qwen-for-PIC24/blob/main/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
from unsloth import FastLanguageModel
import torch

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Qwen2.5-7B-Instruct",
    max_seq_length=2048,
    load_in_4bit= True,
    dtype=None
)

==((====))==  Unsloth 2025.1.5: Fast Qwen2 patching. Transformers: 4.47.1.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/266 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.51k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/632 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing= "unsloth",
    random_state= 3047,
    use_rslora= False,
    loftq_config= None,
)

Unsloth 2025.1.5 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
import pandas as pd
from datasets import Dataset
alpaca_prompt = """You are a PIC24HJ32GP202 microcontroller expert answering student instruction. Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}
"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    outputs = examples["response"]
    texts = []

    for instruction, output in zip(instructions, outputs):
        # Format the prompt without the input field
        text = alpaca_prompt.format(instruction, output) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

from datasets import load_dataset

file_path = "data.csv"
dataframe = pd.read_csv(file_path)
dataset = Dataset.from_pandas(dataframe)

# dataset = load_dataset("data.csv", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)



Map:   0%|          | 0/634 [00:00<?, ? examples/s]

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = 2048,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 50,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Map (num_proc=2):   0%|          | 0/634 [00:00<?, ? examples/s]

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
5.67 GB of memory reserved.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 634 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 50
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss
1,2.854300
2,2.378100
3,2.089800
4,2.532000
5,2.053300
6,1.895500
7,1.376500
8,1.370500
9,1.164300
10,1.037400


In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Hello   ", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True)
decode = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(decode[0])
#assume there is a system input to pin A0 with a wave with unknown frequency and the system will check if frequency bigger than 1kh or not if yes it will giv 1 as output else it will do nothing .output on Pin B3, measure the frequency input

You are a PIC24HJ32GP202 microcontroller expert answering student instruction. Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Hello   

### Response:

Hello! How can I assist you today?



In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "give a sample code of defining input and output on a specific pin and connect the output to LED and set a timer to toggle that ", # instruction
        "", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 2048, use_cache = True)
decode = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(decode[0])
#assume there is a system input to pin A0 with a wave with unknown frequency and the system will check if frequency bigger than 1kh or not if yes it will giv 1 as output else it will do nothing .output on Pin B3, measure the frequency input

You are a PIC24HJ32GP202 microcontroller expert answering student instruction. Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
give a sample code of defining input and output on a specific pin and connect the output to LED and set a timer to toggle that 

### Response:

```
c void main(void) {
    TRISBbits.TRISB0 = 0; // Set RB0 as output
    LATBbits.LATB0 = 1;   // Turn on LED
    T1CON = 0x0001;       // Timer1 configuration
    PR1 = 50000 - 1;      // Set period
    T1IF = 0;             // Clear interrupt flag
    _T1IP = 2;            // Set priority level
    _T1IE = 1;            // Enable interrupt
    _T1ON = 1;            // Start timer
    while (1) {
        if (T1IF == 1) {
            LATBbits.LATB0 = !LATBbits.LATB0; // Toggle LED
            T1IF = 0;                          // Clear interrupt flag
        }
    }
}
```



In [ ]:
model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 9.9G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.65 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


  0%|          | 0/48 [00:00<?, ?it/s]
We will save to Disk and not RAM now.
100%|██████████| 48/48 [03:51<00:00,  4.82s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00006.bin...
Unsloth: Saving model/pytorch_model-00002-of-00006.bin...
Unsloth: Saving model/pytorch_model-00003-of-00006.bin...
Unsloth: Saving model/pytorch_model-00004-of-00006.bin...
Unsloth: Saving model/pytorch_model-00005-of-00006.bin...
Unsloth: Saving model/pytorch_model-00006-of-00006.bin...
Done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!ls /content


data.csv  huggingface_tokenizers_cache	model	 sample_data
drive	  llama.cpp			outputs  _unsloth_sentencepiece_temp


In [ ]:
import shutil

# Replace '/content/model' with the path to your model folder
# Replace '/content/drive/MyDrive/model' with the desired path in your Drive
shutil.copytree('/content/model', '/content/drive/MyDrive/modelOllama/model')


'/content/drive/MyDrive/modelOllama/model'

In [ ]:
!ls /content/drive/MyDrive/model


ls: cannot access '/content/drive/MyDrive/model': No such file or directory


In [ ]:
model.save_pretrained_gguf("model", tokenizer, quantization_method="q4_k_m")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.5G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.85 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 50%|█████     | 14/28 [00:01<00:01, 11.38it/s]
We will save to Disk and not RAM now.
100%|██████████| 28/28 [01:24<00:00,  3.03s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting qwen2 model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into f16 GGUF format.
The output location will be /content/model/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin'
INFO:hf-to-gguf:token_embd.weight,         torch.float16 --> F16, shape = {3584

In [ ]:
print(tokenizer._ollama_modelfile)

AttributeError: Qwen2TokenizerFast has no attribute _ollama_modelfile